In [6]:
import json
from datetime import datetime
import pandas as pd
# # 오늘 날짜를 포함한 파일명 생성
# today_str = datetime.now().strftime("%Y%m%d_%H%M")
# file_name = f"proposal_{today_str}.json"

# # JSON 파일로 저장
# with open(file_name, "w", encoding="utf-8") as f:
#     json.dump(slides_dict, f, indent=2, ensure_ascii=False)

# print(f"✅ JSON 저장 완료: {file_name}")


최신 proposal JSON 파일을 dictionary로 읽기

In [3]:
import os
import json
from datetime import datetime

# 1. JSON 파일들이 있는 디렉토리 경로
json_dir = "DB/proposal/json"

# 2. 파일명 패턴: proposal_YYYYMMDD_HHMM.json
def extract_datetime_from_filename(filename):
    try:
        # "proposal_20250606_1414.json" → datetime object
        base = filename.replace("proposal_", "").replace(".json", "")
        return datetime.strptime(base, "%Y%m%d_%H%M")
    except:
        return None

# 3. JSON 파일 목록 중 최신 파일 찾기
json_files = [
    f for f in os.listdir(json_dir)
    if f.startswith("proposal_") and f.endswith(".json")
]

if not json_files:
    raise FileNotFoundError("❌ proposal JSON 파일이 존재하지 않습니다.")

# 최신 파일 선택
latest_file = max(json_files, key=lambda f: extract_datetime_from_filename(f))
latest_path = os.path.join(json_dir, latest_file)

# 4. 파일 읽어서 dictionary로 변환
with open(latest_path, "r", encoding="utf-8") as f:
    proposal_dict = json.load(f)

print(f"✅ 최신 파일 로드 완료: {latest_file}")
# 필요한 경우 아래 라인으로 딕셔너리 확인


✅ 최신 파일 로드 완료: proposal_20250615_2246.json


In [9]:
from datetime import datetime
import os
# 데이터프레임으로 변환
rows = []
for slide_id, slide_data in proposal_dict.items():
    research_results = slide_data.get("research_results", {})
    for question, result in research_results.items():
        rows.append({
            "slide_id": slide_id,
            "research_question": question,
            "research_summary": result.get("content", ""),
            "urls": result.get("urls", [])
        })

df = pd.DataFrame(rows)
# 오늘 날짜를 YYYYMMDD 형식으로 지정
today_str = datetime.today().strftime('%Y%m%d')

# 저장 경로 설정
save_dir = 'DB/proposal/research_results'
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, f'research_results_{today_str}.xlsx')

# 엑셀로 저장
df.to_excel(save_path, index=False)

In [10]:
def clean_slide(slide):
    # 특정 키 제거
    slide.pop("needs_research", None)
    slide.pop("research_results", None)
    
    # elements 안쪽에 urls 있을 수 있으므로 탐색
    if "elements" in slide:
        for key, value in slide["elements"].items():
            if isinstance(value, dict) and "urls" in value:
                value.pop("urls", None)
    return slide

# 모든 슬라이드에 대해 정리
for slide_id in proposal_dict:
    proposal_dict[slide_id] = clean_slide(proposal_dict[slide_id])

In [11]:
proposal_dict

{'cover_page': {'elements': {'Title': 'AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025. 05. 15',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 컨설팅 로고'}},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '제안서의 주요 섹션과 슬라이드를 한눈에 파악할 수 있도록 구성합니다.',
  'elements': {'Title': '목차',
   'SectionList': ['1. 제안 개요',
    '2. 프로젝트 이해',
    '3. 고객 요구사항 요약',
    '4. 시장 분석 및 트렌드',
    '5. 경쟁 벤치마킹',
    '6. 솔루션 개요',
    '7. 기술 사양',
    '8. 구현 계획',
    '9. 기대 효과',
    '10. 팀 소개',
    '11. 차별화 요소',
    '12. Q&A',
    '13. 부록']}},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': 'AI 모델 학습 및 평가 자동화 프로세스의 중요성과 EY 컨설팅의 차별화된 접근 방식을 요약합니다.',
  'elements': {'Title': '제안 개요',
   'MiddleText': 'AI 모델 학습 및 평가 자동화의 필요성과 EY 컨설팅의 솔루션 개요',
   'SummaryPoints': ['AI 기술 수요 증가에 따른 체계적 데이터 관리 및 학습 인프라 구축 필요',
    '대규모 비정형 데이터의 신속한 정제 및 가공 체계 필요',
    '자동화된 A

In [62]:
pptx_path = "DB/rfp_to_proposal.pptx"
output_path = "DB/proposal/final_ppt/full_generated_proposal_v1.pptx"
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우의 기본 한글 폰트


In [63]:
from pathlib import Path
from pptx import Presentation
from pptx.dml.color import RGBColor
import matplotlib.pyplot as plt
from matplotlib import font_manager
import os
import re

In [68]:
import os
import re
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib import font_manager
from pptx import Presentation
from pptx.dml.color import RGBColor

# 📌 한글 폰트 설정 (Windows 전용: Malgun Gothic)
font_path = "C:/Windows/Fonts/malgun.ttf"
if os.path.exists(font_path):
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    plt.rcParams["font.family"] = font_name

# 📌 그래프 또는 테이블 삽입 함수
def insert_graph_or_table(slide, placeholder, graph_data):
    from matplotlib import pyplot as plt

    graph_type = graph_data.get("graph_type", "")
    table_data = graph_data.get("table_data", [])

    chart_dir = Path("./DB/charts")
    chart_dir.mkdir(parents=True, exist_ok=True)
    chart_path = chart_dir / "temp_chart.png"

    if graph_type == "Bar Chart":
        categories = [row[0] for row in table_data[1:]]
        values_raw = [row[1] for row in table_data[1:]]

        def parse_value(v):
            match = re.search(r"[\d.]+", v)
            return float(match.group()) if match else 0.0

        values = [parse_value(v) for v in values_raw]

        fig, ax = plt.subplots()
        ax.bar(categories, values)
        ax.set_title(graph_data.get("data_source", ""))
        fig.tight_layout()

        plt.savefig(chart_path)
        plt.close()

        for shape in slide.shapes:
            if shape.has_text_frame and shape.text.strip() == placeholder:
                left, top, width, height = shape.left, shape.top, shape.width, shape.height
                slide.shapes._spTree.remove(shape._element)
                slide.shapes.add_picture(str(chart_path), left, top, width, height)
                break
    else:
        rows, cols = len(table_data), len(table_data[0])
        for shape in slide.shapes:
            if shape.has_text_frame and shape.text.strip() == placeholder:
                left, top, width, height = shape.left, shape.top, shape.width, shape.height
                slide.shapes._spTree.remove(shape._element)
                table_shape = slide.shapes.add_table(rows, cols, left, top, width, height).table
                for i in range(rows):
                    for j in range(cols):
                        cell = table_shape.cell(i, j)
                        cell.text = table_data[i][j]
                        if i == 0:
                            cell.fill.solid()
                            cell.fill.fore_color.rgb = RGBColor(230, 230, 230)
                break

# 📌 텍스트 삽입 함수
def insert_text(slide, target_text, replacement, bullet=False):
    for shape in slide.shapes:
        if shape.has_text_frame and shape.text.strip() == target_text:
            shape.text_frame.clear()
            if isinstance(replacement, list) or bullet:
                for item in replacement:
                    p = shape.text_frame.add_paragraph()
                    p.text = f"• {item}"
            else:
                shape.text = replacement

# 📌 슬라이드별 내용 삽입 함수
def fill_cover_page(slide, elements):
    insert_text(slide, "Title", elements.get("Title", ""))
    insert_text(slide, "Subtitle", elements.get("Subtitle", ""))
    insert_text(slide, "ProjectDate", elements.get("ProjectDate", ""))
    insert_text(slide, "PreparedBy", elements.get("PreparedBy", ""))
    insert_text(slide, "Log", elements.get("Logo", ""))

def fill_executive_summary(slide, elements):
    insert_text(slide, "Title", elements.get("Title", ""))
    insert_text(slide, "SummaryPoints", elements.get("SummaryPoints", []), bullet=True)
    insert_text(slide, "ClientValue", elements.get("ClientValue", ""))
    insert_text(slide, "GraphLeftdescription", elements.get("GraphLeft", {}).get("description", ""))
    insert_graph_or_table(slide, "GraphLeft", elements.get("GraphLeft", {}))

def fill_project_understanding(slide, elements):
    insert_text(slide, "Title", elements.get("Title", ""))
    insert_text(slide, "MiddleText", elements.get("MiddleText", ""))
    insert_text(slide, "BackgroundIssues", elements.get("BackgroundIssues", []), bullet=True)
    insert_text(slide, "GraphLeftdescription", elements.get("GraphLeft", {}).get("description", ""))
    insert_graph_or_table(slide, "GraphLeft", elements.get("GraphLeft", {}))

def fill_client_needs_summary(slide, elements):
    insert_text(slide, "Title", elements.get("Title", ""))
    insert_text(slide, "MiddleText", elements.get("MiddleText", ""))
    insert_text(slide, "BulletPoints", elements.get("BulletPoints", []), bullet=True)
    insert_text(slide, "GraphLeftdescription", elements.get("GraphLeft", {}).get("description", ""))
    insert_graph_or_table(slide, "GraphLeft", elements.get("GraphLeft", {}))
    insert_graph_or_table(slide, "NeedsMatrix", {"table_data": elements.get("NeedsMatrix", [])})

# ✅ apply_to_ppt 함수: 전체 흐름 실행
def apply_to_ppt(prs: Presentation, proposal_dict: dict):
    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            text = shape.text.strip()
            if text == "cover_page":
                fill_cover_page(slide, proposal_dict.get("cover_page", {}).get("elements", {}))
            elif text == "executive_summary":
                fill_executive_summary(slide, proposal_dict.get("executive_summary", {}).get("elements", {}))
            elif text == "project_understanding":
                fill_project_understanding(slide, proposal_dict.get("project_understanding", {}).get("elements", {}))
            elif text == "client_needs_summary":
                fill_client_needs_summary(slide, proposal_dict.get("client_needs_summary", {}).get("elements", {}))

# 🎯 전체 함수 작성 완료
"✅ 전체 코드 완성: apply_to_ppt(prs, proposal_dict) 로 실행 가능"


'✅ 전체 코드 완성: apply_to_ppt(prs, proposal_dict) 로 실행 가능'

In [70]:
from pptx import Presentation

# 1. 템플릿 PPT 불러오기
prs = Presentation("DB/rfp_to_proposal.pptx")

# 2. proposal_dict 내용 채워서 적용
apply_to_ppt(prs, proposal_dict)

# 3. 결과 저장 (경로 자동 생성 필요 시 os.makedirs 사용)
output_path = "DB/proposal/final_result/full_generated_proposal_v1.pptx"
os.makedirs(Path(output_path).parent, exist_ok=True)
prs.save(output_path)


C:\Users\HyunJunLee\AppData\Local\Temp\ipykernel_6796\1770976355.py:39: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) Malgun Gothic.
  fig.tight_layout()
C:\Users\HyunJunLee\AppData\Local\Temp\ipykernel_6796\1770976355.py:41: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) Malgun Gothic.
  plt.savefig(chart_path)


IndexError: list index out of range